In [ ]:
# 数据预处理

# 词向量
word_vectors_cache = ""
from torchnlp.word_to_vector import GloVe, FastText
# GloVe: 参数name, 可取['6B', '42B', '840B', 'twitter.27B']; 参数cache, 设置词向量缓存地址
	# 若name为'6B'或'twitter.27B', 还有一个参数dim, 即确定获取的词向量维数
word_vectors = GloVe(name='6B', dim=300, cache=word_vectors_cache) # 'GloVe_6B', 'GloVe_42B', 'GloVe_840B', 'GloVe_twitter.27B'
# FastText: 参数为language='en', cache同上
	# 会出现问题: Calling FastText() results to HTTP Error 301: Moved Permanently
	# 可改用: from torchtext.vocab import FastText
word_vectors = FastText(language='en', cache=word_vectors_cache) # 会出现问题: urllib.error.HTTPError: HTTP Error 403: Forbidden

# 编码
from torchnlp.encoders.text import SpacyEncoder # 第一个参数为文本字符串列表 
text_corpus = []
encoder = SpacyEncoder(text_corpus, min_occurrences=3)
encoder.encode() # 索引化, 得到字符串的编码整数序列 
encoder.vocab # 得到词汇列表 
encoder.decode() # 输入字符串编码序列得到对应的字符串 
encoder.vocab_size # 词汇表大小
from torchnlp.encoders.text.default_reserved_tokens import DEFAULT_EOS_INDEX, DEFAULT_SOS_INDEX, DEFAULT_UNKNOWN_INDEX, DEFAULT_PADDING_INDEX
print(DEFAULT_EOS_INDEX, DEFAULT_UNKNOWN_INDEX, DEFAULT_PADDING_INDEX, DEFAULT_SOS_INDEX) # 2 1 0 3 
from torchnlp.encoders.text.text_encoder import stack_and_pad_tensors # 按一个batch中最长的进行pad, 不足的以pad值填充
# 返回SequenceBatch: pad好的张量以及张量的原始长度 

In [ ]:
# 数据集

from torchnlp.datasets.dataset import Dataset
train_set = Dataset(examples=[{},...,{}])
train_set.columns.add('index')

# 迭代
from torchnlp.utils import datasets_iterator # 用在循环中, 对数据进行迭代
test_set = None
text_corpus = [row['text'] for row in datasets_iterator(train_set, test_set)]
# 一般更倾向于使用itertools.chain

# 采样器
from torchnlp.samplers import BucketBatchSampler
dataset = None
BucketBatchSampler(dataset, batch_size=32, drop_last=False, sort_key=lambda r: len(r['text'])) # 按长度从大到小排列, 然后以批大小重新组合数据